# Model Training
This notebook is running a convolutional neural network model on top of downloaded furniture images.

In [ ]:
# Necessary Packages
from io import BytesIO
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load pre-trained MobileNetV2 model without the top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Define a new model that outputs the features
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def extract_features(image_path, model):
    img_array = load_and_preprocess_image(image_path)
    features = model.predict(img_array)
    return features.flatten()


In [ ]:
# Running feature extraction on downloaded images
dataset_path = "<Path to folder>"
dataset_features = []
for filename in os.listdir(dataset_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(dataset_path, filename)
        features = extract_features(image_path, model)
        dataset_features.append((filename, features))
print("All Done")


## Example Usage

Running the similarity model on an example image input.

In [ ]:
# Example usage
query_image_path = "<path to folder/example_input.jpg>"

# Extract features for the query image
query_features = extract_features(query_image_path, model)

similarities = []
for filename, features in dataset_features:
    similarity = cosine_similarity([query_features], [features])[0][0]
    similarities.append((filename, similarity))

# Sort the images by similarity score
similarities.sort(key=lambda x: x[1], reverse=True)

## Extract features of all the images on the folder
Running feature extraction on all images based on given folder (Downloaded images).

In [ ]:
# All features of active range images of Sweden
dataset_path = "<Path to folder>"
dataset_features = []
for filename in os.listdir(dataset_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(dataset_path, filename)
        features = extract_features(image_path, model)
        dataset_features.append((filename, features))
print("All Done")

## Export to model for outer usage
Exporting and downloading the model to be able to use it in demo script.

In [ ]:
# Export the model
import joblib
model_file = '<model_name.sav>'
joblib.dump(model, model_file)

In [ ]:
# Save the dataset features to a file using Pickle
# It contains image features of all images
import pickle
with open('image_features.pkl', 'wb') as file:
    pickle.dump(dataset_features, file)